In [1]:
import os
from huggingface_hub import login

hf_token = os.environ['HF_TOKEN']
login(hf_token)

/home/iamjunhee/gemma_test/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
from transformers import AutoProcessor, AutoModelForImageTextToText
import torch

USE_PLAIN = True

if USE_PLAIN:
    model_path = "google/gemma-3-4b-it"
else:
    model_path = "./agricsense.model"

model = AutoModelForImageTextToText.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2"
)

processor = AutoProcessor.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/junhee0110/.agricsense/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:550: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['language_model.lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [4]:
from datasets import load_dataset

DATASET_BASE = "/home/iamjunhee/OhMyData/SpatialBench"
TEST_ITEMS = ["counting", "existence", "positional", "reach", "size"]
DATA_PER_ITEM = 5

get_json_dir = lambda item: os.path.join(DATASET_BASE, "{}.json".format(item))

dataset = [load_dataset("json", data_files=get_json_dir(item), split="train") for item in TEST_ITEMS]

Generating train split: 20 examples [00:00, 1114.47 examples/s]
Generating train split: 40 examples [00:00, 4780.92 examples/s]
Generating train split: 34 examples [00:00, 4701.20 examples/s]
Generating train split: 40 examples [00:00, 6503.05 examples/s]
Generating train split: 40 examples [00:00, 7003.64 examples/s]


In [31]:
def get_image_path(rel_path, is_depth):
    path_split = rel_path.split("/")

    if is_depth:
        path_split[0] += "_d"
        file_name = path_split[1].split(".")
        file_name[-1] = "png"
        path_split[-1] = ".".join(file_name)

    return os.path.join(DATASET_BASE, *path_split) 

In [53]:
import random

def get_sample_from_dataset(dataset, seed, N = 5):
    result = []
    random.seed(seed)
    for item in dataset:
        result += random.sample(list(item), N)

    return result

In [67]:
from pipeline import generate_prompt, generate, generate_prompt_with_context

bench_sample = get_sample_from_dataset(dataset, 5)

prompt = [
    generate_prompt(sample["question"],
                    get_image_path(sample["image"], is_depth = False),
                    None if USE_PLAIN else get_image_path(sample["image"], is_depth = True)
    ) for sample in bench_sample
]

NameError: name 'USE_PLAIN' is not defined

In [ ]:
create_batch = lambda data, batch_size: [data[i:i+batch_size] for i in range (0, len(data), batch_size)]

batched_prompt = create_batch(prompt, 5)
answers = []

for item in batched_prompt:
    generated = generate(item, model, processor, max_new_tokens=100)["generated"]
    answers+=generated

In [ ]:
from copy import deepcopy
import json

bench_result = deepcopy(bench_sample)

for index in range(len(bench_result)):
    bench_result[index]["generated"] = answers[index]

print(json.dump(bench_result))